In [10]:
from functools import reduce

import pandas as pd
import numpy as np

import pickle

In [6]:
preds_path_ls = [
    ('raw 0 shot',   r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\Raw_0sh.bin'),
    ('raw 5 shot',   r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\Raw_5sh.bin'),
    ('LoRA 0 shot',  r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\LoRA-cfg13_0sh.bin'),
    ('LoRA 5 shot',  r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\LoRA-cfg13_5sh.bin'),
    ('GSOFT 0 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg0_0sh.bin'),
    ('GSOFT 5 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg0_5sh.bin'),
]

In [7]:
preds_dfs = {}

for run_name, preds_path in preds_path_ls:
    with open(preds_path, 'rb') as f:
        preds_dfs[run_name] = pickle.load(file=f)

        print(f"{run_name:16}: {preds_dfs[run_name].shape=}")

raw 0 shot      : preds_dfs[run_name].shape=(14042, 6)
raw 5 shot      : preds_dfs[run_name].shape=(14042, 6)
LoRA 0 shot     : preds_dfs[run_name].shape=(14042, 6)
LoRA 5 shot     : preds_dfs[run_name].shape=(14042, 6)
GSOFT 0 shot    : preds_dfs[run_name].shape=(14042, 6)
GSOFT 5 shot    : preds_dfs[run_name].shape=(14042, 6)


In [29]:
preds_dfs['GSOFT 0 shot'].head()

,generated_text,subject,pred,true,corr,category
0,B. 4,abstract_algebra,B,B,1,STEM
1,A. 8,abstract_algebra,A,C,0,STEM
2,We have that x,abstract_algebra,I,D,0,STEM
3,"A. True,",abstract_algebra,A,B,0,STEM
4,B. 6,abstract_algebra,B,B,1,STEM


In [24]:
acc_by_subjects = pd.DataFrame({
    'subject': list(set.union(
        *map(
            lambda df: set(df['subject']),
            preds_dfs.values()
        )
    ))
})

print(f"Subjects: {acc_by_subjects['subject']}")

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['subject', 'corr']].groupby(['subject'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_subjects = acc_by_subjects.merge(
        acc_df,
        left_on='subject',
        right_on='subject',
        how='right'
    )

Subjects: 0                       computer_security
1                     high_school_physics
2                                 anatomy
3                college_computer_science
4               high_school_world_history
5                       us_foreign_policy
6                               astronomy
7                               nutrition
8                  electrical_engineering
9                              philosophy
10                      college_chemistry
11                  professional_medicine
12             high_school_macroeconomics
13                 high_school_statistics
14                        college_physics
15                        business_ethics
16                           econometrics
17                  high_school_chemistry
18                    high_school_biology
19           high_school_computer_science
20                       security_studies
21    high_school_government_and_politics
22                     conceptual_physics
23                 high_

In [25]:
acc_by_subjects

,subject,raw 0 shot,raw 5 shot,LoRA 0 shot,LoRA 5 shot,GSOFT 0 shot,GSOFT 5 shot
0,abstract_algebra,0.350000,0.310000,0.400000,0.330000,0.300000,0.310000
1,anatomy,0.674074,0.666667,0.622222,0.666667,0.637037,0.622222
2,astronomy,0.750000,0.723684,0.710526,0.723684,0.723684,0.684211
3,business_ethics,0.660000,0.670000,0.620000,0.660000,0.720000,0.690000
4,clinical_knowledge,0.716981,0.747170,0.735849,0.750943,0.754717,0.750943
5,college_biology,0.715278,0.756944,0.791667,0.763889,0.743056,0.770833
6,college_chemistry,0.450000,0.460000,0.460000,0.480000,0.490000,0.480000
7,college_computer_science,0.470000,0.570000,0.480000,0.580000,0.410000,0.570000
8,college_mathematics,0.350000,0.350000,0.330000,0.370000,0.340000,0.340000
9,college_medicine,0.618497,0.641618,0.624277,0.647399,0.664740,0.664740


In [30]:
acc_by_categories = pd.DataFrame({
    'category': list(set.union(
        *map(
            lambda df: set(df['category']),
            preds_dfs.values()
        )
    ))
})

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['category', 'corr']].groupby(['category'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_categories = acc_by_categories.merge(
        acc_df,
        left_on='category',
        right_on='category',
        how='right'
    )

In [39]:
acc_by_categories.loc[:, list(filter(
    lambda s: ('5 shot' not in s),
    acc_by_categories.columns
))]

,category,raw 0 shot,LoRA 0 shot,GSOFT 0 shot
0,STEM,0.525182,0.542744,0.534129
1,humanities,0.527099,0.568757,0.557917
2,"other (business, health, misc.)",0.706663,0.705737,0.719309
3,social sciences,0.721807,0.746831,0.756906


In [40]:
acc_by_categories.loc[:, list(filter(
    lambda s: ('0 shot' not in s),
    acc_by_categories.columns
))]

,category,raw 5 shot,LoRA 5 shot,GSOFT 5 shot
0,STEM,0.546720,0.562293,0.552353
1,humanities,0.603826,0.595537,0.607439
2,"other (business, health, misc.)",0.724244,0.718075,0.726095
3,social sciences,0.758531,0.760806,0.765356


In [41]:
total_res = pd.DataFrame(columns=['run_name', 'accuracy', 'correctness'])

for run_name, preds_df in preds_dfs.items():
    total_accuracy = preds_df['corr'].mean()
    total_correctness = (preds_df['pred'] != 'I').mean()

    total_res = pd.concat([
        total_res,
        pd.DataFrame({
            'run_name': [run_name],
            'accuracy': [total_accuracy],
            'correctness': [total_correctness],
        })
    ])

C:\Users\Vladimir\AppData\Local\Temp\ipykernel_5496\2066953499.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_res = pd.concat([


In [42]:
total_res

,run_name,accuracy,correctness
0,raw 0 shot,0.610810,0.997151
0,raw 5 shot,0.653255,0.999217
0,LoRA 0 shot,0.633813,0.995656
0,LoRA 5 shot,0.652898,1.000000
0,GSOFT 0 shot,0.633670,0.992095
0,GSOFT 5 shot,0.657599,1.000000
